In [1]:
%load_ext sql

In [2]:
%sql sqlite:///myfile.db

In [78]:
%%sql
SELECT year_of_releas,
       genre,
	   COUNT(*)
FROM movies
group by year_of_releas, genre
order by COUNT(*) Desc limit 5

 * sqlite:///myfile.db
Done.


year_of_releas,genre,COUNT(*)
2006,Drama,6
2004,Drama,4
2007,Drama,4
2011,Drama,4
2002,Comedy,3


In [80]:
%%sql
-- Augment the records of movie rentals with information about movies and customers
SELECT *
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id limit 4;

 * sqlite:///myfile.db
Done.


renting_id,customer_id,movie_id,rating,date_renting,movie_id_1,title,genre,runtime,year_of_releas,renting_price,customer_id_1,name,country,gender,date_of_birth,date_account_start
1,41,8,None,2018-10-09,8,Waking Up in Reno,Comedy,91,2002,2.59,41,Zara Mitchell,Great Britan,female,1994-07-08,2017-06-12
2,10,29,10.0,2017-03-01,29,Two for the Money,Drama,122,2005,2.79,10,Arnout Veenhuis,Belgium,male,1984-07-26,2017-01-28
3,108,45,4.0,2018-06-08,45,Burn After Reading,Drama,96,2008,2.39,108,Saúl Tafoya Meraz,Spain,male,1992-05-15,2017-03-13
4,39,66,8.0,2018-10-22,66,The Hunger Games,Drama,142,2012,1.59,39,Amy Haynes,Great Britan,female,1975-07-28,2018-01-19


In [85]:
%%sql
-- Calculate the average rating for each country
SELECT 
	c.country, 
	round(AVG(r.rating),3) as avg_rating
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY c.country 
order by avg(r.rating) desc 
limit 4;

 * sqlite:///myfile.db
Done.


country,avg_rating
Belgium,8.9
Slovenia,8.432
Italy,8.138
Poland,8.121


In [88]:
%%sql
SELECT 
	c.country, 
	m.genre, 
	AVG(r.rating) AS avg_rating -- Calculate the average rating 
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY c.country, m.genre limit 6; -- For all aggregation levels of country and genre

 * sqlite:///myfile.db
Done.


country,genre,avg_rating
None,Animation,10.0
None,Comedy,None
None,Drama,6.0
None,Science Fiction & Fantasy,None
Austria,Comedy,8.0
Austria,Drama,6.0


In [89]:
%%sql
-- Count the total number of customers, the number of customers for each country, and the number of female and male customers for each country
SELECT country,
       gender,
	   COUNT(*)
FROM customers
GROUP BY ROLLUP (country, gender)
ORDER BY country, gender; -- Order the result by country and gender

 * sqlite:///myfile.db
(sqlite3.OperationalError) no such function: ROLLUP
[SQL: -- Count the total number of customers, the number of customers for each country, and the number of female and male customers for each country
SELECT country,
       gender,
	   COUNT(*)
FROM customers
GROUP BY ROLLUP (country, gender)
ORDER BY country, gender; -- Order the result by country and gender]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [90]:
%%sql
SELECT 
    CASE WHEN country IS NULL AND gender IS NULL THEN 'All' ELSE country END AS country,
    CASE WHEN gender IS NULL THEN 'All' ELSE gender END AS gender,
    COUNT(*) AS count
FROM customers
GROUP BY country, gender WITH ROLLUP
HAVING (country IS NOT NULL OR gender IS NOT NULL)
ORDER BY country ASC NULLS LAST, gender ASC NULLS LAST;

 * sqlite:///myfile.db
(sqlite3.OperationalError) near "WITH": syntax error
[SQL: SELECT 
    CASE WHEN country IS NULL AND gender IS NULL THEN 'All' ELSE country END AS country,
    CASE WHEN gender IS NULL THEN 'All' ELSE gender END AS gender,
    COUNT(*) AS count
FROM customers
GROUP BY country, gender WITH ROLLUP
HAVING (country IS NOT NULL OR gender IS NOT NULL)
ORDER BY country ASC NULLS LAST, gender ASC NULLS LAST;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [93]:
%%sql
-- Join the tables
SELECT *
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id limit 5;

 * sqlite:///myfile.db
Done.


renting_id,customer_id,movie_id,rating,date_renting,movie_id_1,title,genre,runtime,year_of_releas,renting_price,customer_id_1,name,country,gender,date_of_birth,date_account_start
1,41,8,None,2018-10-09,8,Waking Up in Reno,Comedy,91,2002,2.59,41,Zara Mitchell,Great Britan,female,1994-07-08,2017-06-12
2,10,29,10.0,2017-03-01,29,Two for the Money,Drama,122,2005,2.79,10,Arnout Veenhuis,Belgium,male,1984-07-26,2017-01-28
3,108,45,4.0,2018-06-08,45,Burn After Reading,Drama,96,2008,2.39,108,Saúl Tafoya Meraz,Spain,male,1992-05-15,2017-03-13
4,39,66,8.0,2018-10-22,66,The Hunger Games,Drama,142,2012,1.59,39,Amy Haynes,Great Britan,female,1975-07-28,2018-01-19
5,104,15,7.0,2019-03-18,15,Something's Gotta Give,Comedy,128,2003,1.69,104,Dimitri Zambrano Morales,Spain,male,1992-06-21,2017-04-12


In [96]:
%%sql
SELECT 
	c.country, -- Select country
	m.genre, -- Select genre
    AVG(r.rating), -- Average ratings
	COUNT(*) -- Count number of movie rentals
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY c.country, m.genre -- Aggregate for each country and each genre
ORDER BY c.country, m.genre limit 10;

 * sqlite:///myfile.db
Done.


country,genre,AVG(r.rating),COUNT(*) -- Count number of movie rentals
None,Animation,10.0,1
None,Comedy,None,1
None,Drama,6.0,2
None,Science Fiction & Fantasy,None,1
Austria,Comedy,8.0,1
Austria,Drama,6.0,2
Austria,Mystery & Suspense,None,1
Austria,Science Fiction & Fantasy,6.666666666666667,3
Belgium,Comedy,None,1
Belgium,Drama,9.166666666666666,15


In [99]:
%%sql
-- Group by each county and genre with OLAP extension
SELECT 
	c.country, 
	m.genre, 
	AVG(r.rating) AS avg_rating, 
	COUNT(*) AS num_rating
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY c.country, m.genre
ORDER BY c.country, m.genre limit 6;

 * sqlite:///myfile.db
Done.


country,genre,avg_rating,num_rating
None,Animation,10.0,1
None,Comedy,None,1
None,Drama,6.0,2
None,Science Fiction & Fantasy,None,1
Austria,Comedy,8.0,1
Austria,Drama,6.0,2


In [ ]:
# SELECT 
#   r.customer_id, 
#   m.genre, 
#   AVG(r.rating), 
#   COUNT(*)
# FROM renting AS r
# LEFT JOIN movies AS m
# ON r.movie_id = m.movie_id
# GROUP BY GROUPING SETS ((r.customer_id, m.genre), (r.customer_id), ());

In [105]:
%%sql
SELECT 
  r.customer_id, 
  m.genre, 
  AVG(r.rating), 
  COUNT(*)
FROM renting AS r
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
GROUP BY r.customer_id, m.genre, r.customer_id 
order by 
  COUNT(*) desc limit 5;

 * sqlite:///myfile.db
Done.


customer_id,genre,AVG(r.rating),COUNT(*)
21,Drama,6.0,8
92,Drama,7.75,8
108,Drama,7.8,8
113,Drama,7.0,8
26,Drama,8.0,7


In [106]:
%%sql
SELECT 
	nationality, -- Select nationality of the actors
    gender, -- Select gender of the actors
    COUNT(*) -- Count the number of actors
FROM actors
GROUP BY GROUPING SETS ((nationality), (gender), ()); -- Use the correct GROUPING SETS operation

 * sqlite:///myfile.db
(sqlite3.OperationalError) near "SETS": syntax error
[SQL: SELECT 
	nationality, -- Select nationality of the actors
    gender, -- Select gender of the actors
    COUNT(*) -- Count the number of actors
FROM actors
GROUP BY GROUPING SETS ((nationality), (gender), ()); -- Use the correct GROUPING SETS operation]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [109]:
%%sql
SELECT 
	c.country, -- Select country, gender and rating
	c.gender,
	r.rating
FROM renting AS r
LEFT JOIN customers AS c -- Use the correct join
ON r.customer_id = c.customer_id limit 5;

 * sqlite:///myfile.db
Done.


country,gender,rating
Great Britan,female,None
Belgium,male,10.0
Spain,male,4.0
Great Britan,female,8.0
Spain,male,7.0


In [111]:
%%sql
SELECT 
	c.country, 
    c.gender,
	AVG(r.rating) -- Calculate average rating
FROM renting AS r
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY country, gender  -- Order and group by country and gender
ORDER BY country, gender
limit 5;

 * sqlite:///myfile.db
Done.


country,gender,AVG(r.rating) -- Calculate average rating
None,None,8.0
Austria,female,7.0
Austria,male,6.0
Belgium,female,9.125
Belgium,male,8.0


In [112]:
%%sql
SELECT 
	c.country, 
    c.gender,
	AVG(r.rating)
FROM renting AS r
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
-- Report all info from a Pivot table for country and gender
GROUP BY GROUPING SETS ((country, gender), (country), (gender), ());

 * sqlite:///myfile.db
(sqlite3.OperationalError) near "SETS": syntax error
[SQL: SELECT 
	c.country, 
    c.gender,
	AVG(r.rating)
FROM renting AS r
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
-- Report all info from a Pivot table for country and gender
GROUP BY GROUPING SETS ((country, gender), (country), (gender), ());]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [118]:
%%sql
SELECT *
FROM renting AS r
LEFT JOIN movies AS m -- Augment the table with information about movies
ON r.movie_id = m.movie_id limit 4;

 * sqlite:///myfile.db
Done.


renting_id,customer_id,movie_id,rating,date_renting,movie_id_1,title,genre,runtime,year_of_releas,renting_price
1,41,8,None,2018-10-09,8,Waking Up in Reno,Comedy,91,2002,2.59
2,10,29,10.0,2017-03-01,29,Two for the Money,Drama,122,2005,2.79
3,108,45,4.0,2018-06-08,45,Burn After Reading,Drama,96,2008,2.39
4,39,66,8.0,2018-10-22,66,The Hunger Games,Drama,142,2012,1.59


In [119]:
%%sql
SELECT *
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
WHERE r.movie_id IN ( -- Select records of movies with at least 4 ratings
	SELECT movie_id
	FROM renting
	GROUP BY movie_id
	HAVING COUNT(rating) >= 4)
AND r.date_renting >= '2018-04-01' limit 5; -- Select records of movie rentals since 2018-04-01

 * sqlite:///myfile.db
Done.


renting_id,customer_id,movie_id,rating,date_renting,movie_id_1,title,genre,runtime,year_of_releas,renting_price
3,108,45,4.0,2018-06-08,45,Burn After Reading,Drama,96,2008,2.39
4,39,66,8.0,2018-10-22,66,The Hunger Games,Drama,142,2012,1.59
6,50,71,7.0,2018-10-09,71,Rio 2,Animation,101,2014,1.79
8,73,65,10.0,2018-06-05,65,Ghost Rider: Spirit of Vengeance,Action & Adventure,96,2012,1.79
12,52,65,10.0,2018-06-29,65,Ghost Rider: Spirit of Vengeance,Action & Adventure,96,2012,1.79


In [120]:
%%sql
SELECT m.genre, -- For each genre, calculate:
	   AVG(r.rating) AS avg_rating, -- The average rating and use the alias avg_rating
	   COUNT(r.rating) AS n_rating, -- The number of ratings and use the alias n_rating
	   COUNT(*) AS n_rentals,     -- The number of movie rentals and use the alias n_rentals
	   COUNT(DISTINCT m.movie_id) AS n_movies -- The number of distinct movies and use the alias n_movies
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
WHERE r.movie_id IN ( 
	SELECT movie_id
	FROM renting
	GROUP BY movie_id
	HAVING COUNT(rating) >= 3)
AND r.date_renting >= '2018-01-01'
GROUP BY m.genre;

 * sqlite:///myfile.db
Done.


genre,avg_rating,n_rating,n_rentals,n_movies
Action & Adventure,8.714285714285714,7,9,2
Animation,7.833333333333333,6,10,2
Art House & International,8.5,4,5,1
Comedy,7.95,20,31,5
Drama,7.748251748251748,143,245,34
Mystery & Suspense,7.428571428571429,7,19,3
Other,8.428571428571429,7,16,2
Science Fiction & Fantasy,8.27659574468085,47,70,10


In [121]:
%%sql
SELECT genre,
	   AVG(rating) AS avg_rating,
	   COUNT(rating) AS n_rating,
       COUNT(*) AS n_rentals,     
	   COUNT(DISTINCT m.movie_id) AS n_movies 
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
WHERE r.movie_id IN ( 
	SELECT movie_id
	FROM renting
	GROUP BY movie_id
	HAVING COUNT(rating) >= 3 )
AND r.date_renting >= '2018-01-01'
GROUP BY genre
order by AVG(rating) desc; -- Order the table by decreasing average rating

 * sqlite:///myfile.db
Done.


genre,avg_rating,n_rating,n_rentals,n_movies
Action & Adventure,8.714285714285714,7,9,2
Art House & International,8.5,4,5,1
Other,8.428571428571429,7,16,2
Science Fiction & Fantasy,8.27659574468085,47,70,10
Comedy,7.95,20,31,5
Animation,7.833333333333333,6,10,2
Drama,7.748251748251748,143,245,34
Mystery & Suspense,7.428571428571429,7,19,3


In [122]:
%%sql
-- Join the tables
SELECT *
FROM renting AS r
LEFT JOIN actsin AS ai
ON ai.movie_id = r.movie_id
LEFT JOIN actors AS a
ON ai.actor_id = a.actor_id limit 5;

 * sqlite:///myfile.db
Done.


renting_id,customer_id,movie_id,rating,date_renting,actsin_id,movie_id_1,actor_id,actor_id_1,name,year_of_birth,nationality,gender
1,41,8,None,2018-10-09,28,8,23,23,Charlize Theron,1975.0,South Africa,female
1,41,8,None,2018-10-09,152,8,103,103,Natasha Richardson,1963.0,British,female
1,41,8,None,2018-10-09,160,8,107,107,Patrick Swayze,1952.0,USA,male
2,10,29,10.0,2017-03-01,5,29,3,3,Al Pacino,1940.0,USA,male
2,10,29,10.0,2017-03-01,143,29,96,96,Matthew McConaughey,1969.0,USA,male


In [123]:
%%sql
SELECT a.nationality,
       a.gender,
	   AVG(r.rating) AS avg_rating, -- The average rating
	   COUNT(r.rating) AS n_rating, -- The number of ratings
	   COUNT(*) AS n_rentals, -- The number of movie rentals
	   COUNT(DISTINCT a.actor_id) AS n_actors -- The number of actors
FROM renting AS r
LEFT JOIN actsin AS ai
ON ai.movie_id = r.movie_id
LEFT JOIN actors AS a
ON ai.actor_id = a.actor_id
WHERE r.movie_id IN ( 
	SELECT movie_id
	FROM renting
	GROUP BY movie_id
	HAVING COUNT(rating) >=4 )
AND r.date_renting >= '2018-04-01'
GROUP BY a.nationality, a.gender;

 * sqlite:///myfile.db
Done.


nationality,gender,avg_rating,n_rating,n_rentals,n_actors
Argentina,male,8.5,4,5,1
Australia,female,8.666666666666666,3,5,1
Australia,male,7.454545454545454,11,17,3
Austria,male,8.5,2,6,1
British,female,7.833333333333333,54,78,3
British,male,8.105263157894736,114,175,9
Canada,female,8.0,5,5,1
Canada,male,8.428571428571429,7,13,2
Ireland,male,7.714285714285714,7,16,2
Israel,female,8.25,4,8,1


In [ ]:
%%sql
